Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

In [22]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
#For google drive
#%%capture
!pip install pyspark
!pip install -U -q PyDrive
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
!tar xf spark-3.0.3-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

In [24]:
import findspark
findspark.init()
from pyspark.sql.functions import udf

## Build Spark Session

In [25]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [26]:
train=spark.read.csv('/content/gdrive/MyDrive/spark/train.csv',header=True,inferSchema=True)
test=spark.read.csv('/content/gdrive/MyDrive/spark/test.csv',header=True,inferSchema=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [27]:
print(type(train))
print(type(test))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


**Show 5 rows.**

In [28]:
train.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display schema for the dataset:**

In [29]:
train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [30]:
train.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [31]:
train.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [32]:
survived=train.select('Survived').where('Survived = 1').count()
not_survived=train.select('Survived').where('Survived = 0').count()

**Display your result:**

In [33]:
print('number of survived',survived)
print('number of people didnt survive',not_survived)

number of survived 342
number of people didnt survive 549


**Can you display your answer in ratio form?(Hint: Use UDF.)**






**Can you get the number of males and females?**


In [34]:
males=train.select('Sex').where('Sex = "male"').count()
females=train.select('Sex').where('Sex = "female"').count()

In [35]:
print(males)
print(females)

577
314


**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column.)

In [36]:
import pyspark.sql.functions as F
male_survived = train.groupBy('Sex').agg(F.mean('Survived'))
male_survived.show()

+------+-------------------+
|   Sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



**Create temporary view PySpark:**

In [37]:
train.createOrReplaceTempView('TRAIN')
test.createOrReplaceTempView('TEST')

**How many people survived, and how many didn't survive? By SQL:**

In [38]:
spark.sql('select count(Survived) From TRAIN where Survived = 1').show()
spark.sql('select count(Survived) From TRAIN where Survived = 0').show()

+---------------+
|count(Survived)|
+---------------+
|            342|
+---------------+

+---------------+
|count(Survived)|
+---------------+
|            549|
+---------------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column.)

**Can you do this via SQL?**

In [39]:
spark.sql('select Sex,avg(Survived) From TRAIN group by Sex').show()


+------+-------------------+
|   Sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



**Display a ratio for p-class:**


In [40]:
spark.sql('select Pclass,count(*) * 100.0/ sum(count(*)) over () as ratio from train group by Pclass').show()

+------+-----------------+
|Pclass|            ratio|
+------+-----------------+
|     1|24.24242424242424|
|     3|55.10662177328844|
|     2|20.65095398428732|
+------+-----------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [41]:
merged_df=train.union(test)

**Display count:**

In [42]:
merged_df.count()

1329

**Temporary view PySpark:**

In [43]:
merged_df.createOrReplaceTempView('merged')

**Can you define the number of null values in each column?**


In [44]:
nullvalues_df=merged_df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in merged_df.columns]
   )
nullvalues_df.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [45]:
nullvalues_df.select(['Age','Cabin','Embarked']).show()

+---+-----+--------+
|Age|Cabin|Embarked|
+---+-----+--------+
|265| 1021|       3|
+---+-----+--------+



## Preprocessing 

**Can you show me the name column from your temporary table?**

In [46]:
spark.sql('select Name from merged').show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



**Run this code:**

In [47]:
combined = merged_df.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display the title and count "Title" column:**

In [48]:
title_df=spark.sql('select Title , count(Title) as counts from combined group by Title')
title_df.show()

+--------+------+
|   Title|counts|
+--------+------+
|     Don|     1|
|    Miss|   257|
|Countess|     2|
|     Col|     4|
|     Rev|     9|
|    Lady|     2|
|  Master|    56|
|     Mme|     1|
|    Capt|     2|
|      Mr|   786|
|      Dr|    11|
|     Mrs|   186|
|     Sir|     2|
|Jonkheer|     2|
|    Mlle|     4|
|   Major|     3|
|      Ms|     1|
+--------+------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [49]:
rare_dict={i[0]:'rare' for i in title_df.select('Title').where('counts < 56').collect()}
for i in title_df.select('Title').collect():
    if i[0] not in rare_dict.keys():
        rare_dict[i[0]]=i[0]
rare_dict

{'Capt': 'rare',
 'Col': 'rare',
 'Countess': 'rare',
 'Don': 'rare',
 'Dr': 'rare',
 'Jonkheer': 'rare',
 'Lady': 'rare',
 'Major': 'rare',
 'Master': 'Master',
 'Miss': 'Miss',
 'Mlle': 'rare',
 'Mme': 'rare',
 'Mr': 'Mr',
 'Mrs': 'Mrs',
 'Ms': 'rare',
 'Rev': 'rare',
 'Sir': 'rare'}

**Run the function:**

In [50]:
def impute_title(title):
    return rare_dict[title]

**Apply the function on "Title" column using UDF:**

In [51]:
from pyspark.sql.types import *
convertUDF = udf(lambda z: impute_title(z))
combined=combined.withColumn('Title',convertUDF(F.col('Title')))
combined.createOrReplaceTempView('combined')

**Display "Title" from table and group by "Title" column:**

In [52]:
spark.sql('select Title from combined group by Title').show()

+------+
| Title|
+------+
|  rare|
|  Miss|
|Master|
|    Mr|
|   Mrs|
+------+



## **Preprocessing Age**

**Based on the age mean, you will fill in the missing age values:**

In [53]:
avgAge=spark.sql('select avg(Age) from combined').collect()
avgAge[0][0]

30.079501879699244

**Fill missing age with age mean:**

In [54]:
combined=combined.fillna(avgAge[0][0],'Age')
combined.select('Age').show()

+------------------+
|               Age|
+------------------+
|              22.0|
|              38.0|
|              26.0|
|              35.0|
|              35.0|
|30.079501879699244|
|              54.0|
|               2.0|
|              27.0|
|              14.0|
|               4.0|
|              58.0|
|              20.0|
|              39.0|
|              14.0|
|              55.0|
|               2.0|
|30.079501879699244|
|              31.0|
|30.079501879699244|
+------------------+
only showing top 20 rows



## **Preprocessing Embarked**

**Select Embarked, count them, order by count Desc, and save in grouped_Embarked variable:**




In [55]:
grouped_Embarked=spark.sql('select Embarked,count(Embarked) as count from combined group by Embarked order by count DESC')

**Show groupped_Embarked:**

In [56]:
grouped_Embarked.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
|       C|  253|
|       Q|  111|
|    null|    0|
+--------+-----+



**Get the groupped_Embarked:** 

In [57]:
max_embarked=grouped_Embarked.select('Embarked').collect()[0][0]
max_embarked

'S'

**Fill missing values with Top 'S' of grouped_Embarked:**

In [58]:
combined=combined.fillna('S','Embarked')
combined.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in combined.columns]
   ).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Title|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0| 1021|       0|    0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+



## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [59]:
@udf(returnType=StringType()) 
def firstLetter(str):
    if str:
        return str[0]

combined=combined.withColumn("Cabin", firstLetter(F.col("Cabin")))

**Show the result:**

In [60]:
combined.select('Cabin').show()

+-----+
|Cabin|
+-----+
| null|
|    C|
| null|
|    C|
| null|
| null|
|    E|
| null|
| null|
| null|
|    G|
|    C|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
+-----+
only showing top 20 rows



**Create the temporary view:**

In [61]:
combined.createOrReplaceTempView('combined')

**Select "Cabin" column, count Cabin column, Group by "Cabin" column, Order By count DESC**  

In [62]:
spark.sql('select Cabin,count(Cabin) as count from combined group by Cabin order by count DESC').show()

+-----+-----+
|Cabin|count|
+-----+-----+
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
| null|    0|
+-----+-----+



**Fill missing values with "U":**

In [63]:
combined=combined.fillna('U','Cabin')


**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

____________________________________________

**Use Pipline to fit and transform:**

In [64]:
combined.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- Title: string (nullable = true)



In [65]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

combined.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string'),
 ('Title', 'string')]

In [66]:
# stringindexer=StringIndexer(inputCols=)

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None) A feature transformer that merges multiple columns into a vector column.**



**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [67]:
X_train, X_test = combined.randomSplit([0.8,0.2],seed=0)
print(f"There are {X_train.count()} rows in the training set, and {X_test.count()} in the test set.")

There are 1060 rows in the training set, and 269 in the test set.


In [80]:
categoricalColumns = [col for (col, dtype) in X_train.dtypes if dtype == "string"]
categoricalColumns.remove("Name")
categoricalColumns.remove("Ticket")
categoricalColumns

['Sex', 'Cabin', 'Embarked', 'Title']

In [81]:
numericColumns = [field for (field,dataType) in X_train.dtypes
              if ((dataType=='double'or dataType=='int')& (field!='Survived'))]
numericColumns.remove('PassengerId')
numericColumns

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [82]:
indexOutputColumns = [x + "_Index" for x in categoricalColumns]
indexOutputColumns

['Sex_Index', 'Cabin_Index', 'Embarked_Index', 'Title_Index']

In [83]:
oheOutputColumns = [x + "_OHE" for x in categoricalColumns]
oheOutputColumns

['Sex_OHE', 'Cabin_OHE', 'Embarked_OHE', 'Title_OHE']

In [84]:
stringindexer = StringIndexer(inputCols=categoricalColumns,
                             outputCols=indexOutputColumns)
ohe = OneHotEncoder(inputCols=indexOutputColumns,
                          outputCols=oheOutputColumns)


In [85]:
vect_assemb_input = oheOutputColumns + numericColumns
vect_assemb_input

['Sex_OHE',
 'Cabin_OHE',
 'Embarked_OHE',
 'Title_OHE',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare']

In [86]:
vecAssembler = VectorAssembler(inputCols=vect_assemb_input,outputCol='features')

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [87]:
rf = RandomForestClassifier(featuresCol='features', labelCol='Survived', predictionCol='prediction')
pipeline =Pipeline(stages = [stringindexer,ohe,vecAssembler,rf])
model=pipeline.fit(X_train)
pred=model.transform(X_test)
pred.select('prediction','Survived','features').show()

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|(20,[0,1,9,11,15,...|
|       0.0|       1|(20,[0,5,9,11,15,...|
|       1.0|       1|(20,[1,12,15,16,1...|
|       0.0|       0|(20,[0,1,9,11,15,...|
|       0.0|       0|(20,[0,1,10,15,16...|
|       0.0|       1|(20,[0,1,10,11,15...|
|       0.0|       0|(20,[0,1,9,11,15,...|
|       1.0|       1|(20,[1,10,12,15,1...|
|       1.0|       0|(20,[1,9,13,15,16...|
|       1.0|       1|(20,[1,9,13,15,16...|
|       0.0|       0|(20,[0,1,10,11,15...|
|       1.0|       1|(20,[3,9,12,15,16...|
|       0.0|       0|(20,[0,2,9,11,15,...|
|       0.0|       1|(20,[0,1,10,14,15...|
|       0.0|       0|(20,[0,1,9,11,15,...|
|       0.0|       0|(20,[0,7,9,11,15,...|
|       0.0|       0|(20,[0,1,9,11,15,...|
|       0.0|       0|(20,[0,1,9,11,15,...|
|       0.0|       0|(20,[0,1,9,11,15,...|
|       1.0|       0|(20,[1,9,12,15,16...|
+----------

**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [88]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='Survived',
                                              predictionCol='prediction',
                                              metricName='accuracy')
print(evaluator.evaluate(pred))

0.8364312267657993


**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
